# Audio Extraction

**Paper:** Automatic identification of Hainan Gibbon calls in passive acoustic recordings

**Authors:** Emmanuel Dufourq, Ian Durbach, James Hansford, Sam Turvey, Amanda Hoepfner

**Year:** March 2020

**Repository:** https://github.com/emmanueldufourq/GibbonClassifier


Extract the gibbon and non-gibbon calls from a particular .wav file. The user specifies the number of seconds to extract to create each of the gibbon and non-gibbon call segments. The labels need to be available (in folder 'Call_Labels'). The code produces two pickle files containing the extracted segments.

For this example, we are processing the file 'HGSM3D_0+1_20160429_051600.wav' which has a sampling rate of 9600 and is downsampled to 4800. The label file 'g_HGSM3D_0+1_20160429_051600.data' contains the labels for the gibbon calls and the label file 'n_HGSM3D_0+1_20160429_051600.data' contains labels for non-gibbon calls. The extracted segments will be 10 seconds long. 

Two pickle files will be saved in the folder 'Pickled_Data', namely: 'g_HGSM3D_0+1_20160429_051600.pkl' and 'n_HGSM3D_0+1_20160429_051600.pkl'. From this raw file 'HGSM3D_0+1_20160429_051600.wav' we obtain 369 gibbon call segments and 1179 non-gibbon call segments each containing 48000 data points (sample rate * segment length, 4800 * 10).

In [1]:
from os import listdir
import random
from shutil import copyfile
import pickle
from Extract_Audio_Helper import *

## FIle name 

Without extension

In [2]:
file_without_extension = 'HGSM3D_0+1_20160429_051600'

## Directories

In [3]:
audio_directory = '../Raw_Data/Train/'
audio_file_name = file_without_extension+'.wav'
timestamp_directory = '../Call_Labels/'
timestamp_file_name = file_without_extension+'.data'
save_location = '../Pickled_Data/'

### Set sample rate

In [4]:
sample_rate = 4800

### Set the size of the audio segments

In [5]:
number_seconds_to_extract = 10

## Read in audio file

In [6]:
librosa_audio, librosa_sample_rate = librosa.load(audio_directory+audio_file_name, 
                                                  sr=sample_rate)

## Read gibbon labelled timestamp file

In [7]:
gibbon_timestamps = read_and_process_gibbon_timestamps(timestamp_directory, 
                                                       'g_'+timestamp_file_name, 
                                               sample_rate, sep=',')

/media/emmanuel/Data2/PycharmProjects/HG/HainanGibbonClassifier/Submit Github/HainanGibbonClassifier/Scipts/Extract_Audio_Helper.py:25: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  gibbon_timestamps['Notes'] = gibbon_timestamps.Notes.str.extract('(\d+)')


In [8]:
gibbon_timestamps

,Start,End,Duration,Type
0,34900800,34910400,2,1
1,35011200,35025600,3,2
2,35203200,35212800,2,1
3,35294400,35313600,4,2
4,35577600,35592000,3,3
5,35678400,35683200,1,2
6,35760000,35764800,1,1
7,35856000,35875200,4,3
8,35976000,36004800,6,3
9,36067200,36081600,3,2


## Read non-gibbon labelled timestamp file

In [9]:
non_gibbon_timestamps = read_and_process_nongibbon_timestamps(timestamp_directory,
                                                              'n_'+timestamp_file_name, 
                                               librosa_sample_rate, sep=',')

## Extract gibbon calls

In [10]:
gibbon_extracted = extract_all_gibbon_calls(librosa_audio, gibbon_timestamps,
                                            number_seconds_to_extract,1, librosa_sample_rate,0)

## Extract background noise

In [11]:
noise_extracted = extract_all_nongibbon_calls(librosa_audio, non_gibbon_timestamps,
                                              number_seconds_to_extract,5, librosa_sample_rate,0)

## Gibbon data extracted

In [12]:
gibbon_extracted.shape

(369, 48000)

## Non-gibbon data extracted

In [13]:
noise_extracted.shape

(1179, 48000)

## Save the extracted data to disk

In [14]:
pickle.dump(gibbon_extracted, open(save_location+'g_'+file_without_extension+'.pkl', "wb" ) )
pickle.dump(noise_extracted, open(save_location+'n_'+file_without_extension+'.pkl', "wb" ) )